In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import numpy as np
np.random.seed(2018)

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [5]:
url='https://drive.google.com/file/d/1XY8kQd7kEivHHmU3jdZAZD8s0Anb8XsP/view'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
trainingDf = pd.read_csv(url)
trainingDf.head()

,score,text,combine
0,3.0,"['good', 'helpfull', 'read', 'book', 'good', '...",good helpfull read book good type thats find d...
1,1.0,"['sadly', 'overprice', 'irrelevant', 'spite', ...",sadly overprice irrelevant spite claim illustr...
2,2.0,"['endless', 'rant', 'howard', 'borrow', 'denni...",endless rant howard borrow dennis miller start...
3,1.0,"['not', 'quite', 'hip', 'really', 'shame', 'ti...",not quite hip really shame time reserch go thi...
4,5.0,"['journey', 'centre', 'earth', 'hey', 'great',...",journey centre earth hey great book absolutely...


In [6]:
import gensim
from gensim import corpora
from pprint import pprint

In [7]:
def myprocess(thisdoc):
    ## Runs on documents (vector of words)
    return([text for text in thisdoc.split()])

In [8]:
docs=trainingDf['combine'].map(myprocess) 
docs[:10]

0    [good, helpfull, read, book, good, type, thats...
1    [sadly, overprice, irrelevant, spite, claim, i...
2    [endless, rant, howard, borrow, dennis, miller...
3    [not, quite, hip, really, shame, time, reserch...
4    [journey, centre, earth, hey, great, book, abs...
5    [long, land, free, repetitive, silly, prose, n...
6    [demon, viewamelia, atwaterrhodes, young, writ...
7    [heartbreakingbut, live, novel, compassionate,...
8    [wait, this, get, book, first, come, finally, ...
9    [awesome, book, absolutely, beautiful, daughte...
Name: combine, dtype: object

In [9]:
from gensim.models import Phrases
import gensim.models

In [10]:
docsBi = trainingDf['combine'].map(myprocess) 
bigram_phrases = Phrases(docsBi, min_count=3, threshold=50) # ignores words with bigrams less than 3
# threshold impacts the number of bigrams found, higher threshold, less bigrams considered

In [11]:
#number of bigrams with frequency over 3
print(len(bigram_phrases.vocab))

2200316


In [12]:
#This code creates a function which replaces the two separate tokens with one bigram if it is in the bigram_phrases corpus
bigram = gensim.models.phrases.Phraser(bigram_phrases)

def make_bigrams(text):
    return(bigram[doc] for doc in text)

In [13]:
print(docs[2])

['endless', 'rant', 'howard', 'borrow', 'dennis', 'miller', 'start', 'book', 'quoti', 'not', 'mean', 'rant', 'here', 'butquot', 'proceed', 'page', 'endless', 'whine', 'horrible', 'law', 'americathe', 'example', 'give', 'about', 'law', 'suffocating', 'america', 'sometimes', 'right', 'mark', 'time', 'not', 'well', 'define', 'less', 'would', 'more', 'idea', 'fix', 'dilemma', 'also', 'goodif', 'live', 'perfect', 'world', 'unfortunately', 'not', 'practical', 'solution', 'message', 'lose', 'amongst', 'myriad', 'word', 'woud', 'not', 'recommend', 'book', 'anyone', 'ted', 'kazinski']


In [14]:
# Here we replace any common word pairs with their bigram. Note 'not', 'good' is replaced with 'not_good'.
docsBi = list(make_bigrams(docs))


In [15]:
docsBi

[['good',
  'helpfull',
  'read',
  'book',
  'good',
  'type',
  'thats',
  'find',
  'difficult',
  'build',
  'muscle',
  'mass',
  'deep',
  'detail',
  'flaw',
  'book',
  'book',
  'doesnt',
  'would',
  'recommend',
  'buy',
  'book',
  'really',
  'lose',
  'book',
  'think',
  'give',
  'head',
  'start'],
 ['sadly',
  'overprice',
  'irrelevant',
  'spite',
  'claim',
  'illustrate',
  'pmo',
  'effective',
  'reducing',
  'cost',
  'project',
  'decrease',
  'time',
  'market',
  'new',
  'product',
  'increase',
  'corporate_profit',
  'ensure',
  'project',
  'success',
  'small',
  'overprice',
  'book',
  'actually',
  'none',
  'thingsa',
  'collection',
  'unrelated',
  'article',
  'write',
  'dry',
  'overly',
  'academic',
  'style',
  'book',
  'totally',
  'fail',
  'address',
  'proposition',
  'project_management',
  'office',
  'pmo',
  'add',
  'value',
  'organization',
  'many',
  'statement',
  'effect',
  'the',
  'pmo',
  'perform',
  'function',
  'asser

In [16]:
dict_LoS = corpora.Dictionary(docs)
dict_LoSBi = corpora.Dictionary(docsBi)

In [17]:
dict_LoS.filter_extremes(no_below=5, no_above=0.5)
dict_LoSBi.filter_extremes(no_below=5, no_above=0.5)

In [18]:
print(dict_LoS)
print(dict_LoSBi)

Dictionary(25541 unique tokens: ['build', 'buy', 'deep', 'detail', 'difficult']...)
Dictionary(30579 unique tokens: ['build', 'buy', 'deep', 'detail', 'difficult']...)


In [19]:
try:
    print("Reading corpus from pickle")
    bow_corpus=pickle.load(open('bow_corpus.pkl', 'rb'))
except FileNotFoundError:
    print("Creating corpus and saving to pickle")
    bow_corpus = [dict_LoS.doc2bow(w) for w in docs]
    pickle.dump(bow_corpus, open('bow_corpus.pkl', 'wb'))
    pickle.dump(dict_LoS, open('dictionary.pkl', 'wb'))
try:
    print("Reading corpus from pickle")
    bow_corpusBi=pickle.load(open('bow_corpusBi.pkl', 'rb'))
except FileNotFoundError:
    print("Creating corpus and saving to pickle")
    bow_corpusBi = [dict_LoSBi.doc2bow(w) for w in docsBi]
    pickle.dump(bow_corpusBi, open('bow_corpusBi.pkl', 'wb'))
    pickle.dump(dict_LoSBi, open('dictionaryBi.pkl', 'wb'))

Reading corpus from pickle
Reading corpus from pickle


In [20]:
bow_doc_16 = bow_corpus[1000]

for i in range(len(bow_doc_16)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_16[i][0], 
                                               dict_LoS[bow_doc_16[i][0]], 
                                                bow_doc_16[i][1]))

Word 15 ("really") appears 1 time.
Word 39 ("completely") appears 1 time.
Word 66 ("half") appears 2 time.
Word 87 ("none") appears 1 time.
Word 91 ("one") appears 1 time.
Word 118 ("time") appears 1 time.
Word 161 ("solution") appears 1 time.
Word 185 ("even") appears 1 time.
Word 201 ("like") appears 1 time.
Word 203 ("must") appears 1 time.
Word 255 ("end") appears 1 time.
Word 289 ("first") appears 1 time.
Word 314 ("different") appears 1 time.
Word 328 ("novel") appears 1 time.
Word 334 ("plot") appears 2 time.
Word 336 ("problem") appears 1 time.
Word 385 ("run") appears 1 time.
Word 389 ("something") appears 2 time.
Word 439 ("enjoy") appears 1 time.
Word 441 ("especially") appears 1 time.
Word 446 ("fiction") appears 1 time.
Word 450 ("himself") appears 1 time.
Word 535 ("move") appears 2 time.
Word 538 ("part") appears 1 time.
Word 572 ("formulaic") appears 1 time.
Word 579 ("least") appears 1 time.
Word 590 ("stuff") appears 1 time.
Word 757 ("difference") appears 1 time.
Wor

In [21]:
# LDA model without considering bigrams
try:
    lda_model=pickle.load(open('lda_model.pkl', 'rb'))
    print("Reading lda_model from pickle")
except FileNotFoundError:
    print("Creating lda_model and saving to pickle")
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=15, id2word=dict_LoS, passes=2, workers=2)
    pickle.dump(lda_model,open('lda_model.pkl','wb'))
    
# LDA model with bigrams included
try:
    lda_modelBi=pickle.load(open('lda_modelBi.pkl', 'rb'))
    print("Reading lda_model from pickle")
except FileNotFoundError:
    print("Creating lda_model and saving to pickle")
    lda_modelBi = gensim.models.LdaMulticore(bow_corpusBi, num_topics=15, id2word=dict_LoSBi, passes=2, workers=2)
    pickle.dump(lda_modelBi,open('lda_modelBi.pkl','wb'))

Reading lda_model from pickle
Reading lda_model from pickle


In [ ]:
pyLDAvis.enable_notebook()
# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, bow_corpus, dict_LoS)
lda_vizBi = gensimvis.prepare(lda_modelBi, bow_corpusBi, dict_LoSBi)

C:\Users\Team Knowhow\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [ ]:
lda_model[bow_corpus[89]]

In [ ]:
lda_model.show_topics(20,7)

In [ ]:
try:
    lda_display=pickle.load(open('lda_display.pkl', 'rb'))
    print("Reading lda_display from pickle")
except FileNotFoundError:
    print("Creating lda_display and saving to pickle")
    lda_display = gensimvis.prepare(lda_model, bow_corpus, 
                                          dict_LoS, mds='mmds')
    pickle.dump(lda_display,open('lda_display.pkl','wb'))
try:
    lda_displayBi=pickle.load(open('lda_displayBi.pkl', 'rb'))
    print("Reading lda_display from pickle")
except FileNotFoundError:
    print("Creating lda_display and saving to pickle")
    lda_displayBi = gensimvis.prepare(lda_modelBi, bow_corpusBi, 
                                          dict_LoSBi, mds='mmds')
    pickle.dump(lda_displayBi,open('lda_displayBi.pkl','wb'))

## LDA MODEL WITHOUT BIGRAMS

In [ ]:
pyLDAvis.display(lda_display, template_type='notebook')

## LDA MODEL WITH BIGRAMS INCLUDED

In [ ]:
pyLDAvis.display(lda_displayBi, template_type='notebook')